In [ ]:
import sys

sys.path.append("../")

# Torch

In [ ]:
import torch
import torch.nn as nn

model = nn.Sequential(nn.Linear(1, 5, bias=False), nn.ReLU(), nn.Linear(5, 1))

B = []
for m in model.modules():
    if m.__class__.__name__.startswith("Linear"):
        B.append(m.bias)
        print(m.bias)

Btot = torch.cat(B, dim=0)

In [ ]:
model[0].bias

In [ ]:
for name, param in model.named_parameters():
    print(name, param)

### buffer

In [ ]:
import torch

mdl = torch.nn.Linear(1, 1)
mdl.register_buffer("free_node", torch.rand((2, 1)))

In [ ]:
torch.cuda.is_available()

In [ ]:
torch.backends.cudnn.is_available()

In [ ]:
for buf in mdl.buffers():
    print(id(buf))

In [ ]:
mdl.bias

## nograd

In [ ]:
import torch


@torch.no_grad()
def foo(x, y):
    x.grad = torch.ones_like(x)
    return x + bar(x, y)


def bar(x, y):
    return x + 2 * y


x, y = torch.rand((2, 1), requires_grad=True), torch.rand((2, 1))
z = foo(x, y)

In [ ]:
@torch.no_grad()
def baz(x: torch.tensor):
    return x.clone().detach().requires_grad_(True)

In [ ]:
baz(torch.tensor([1.0]))

In [ ]:
x = torch.rand((2, 1))

In [ ]:
x.grad = torch.ones_like(x)

In [ ]:
t = torch.tensor([1.0, 2.0, 3.0], requires_grad=False)

t2 = t.clone().detach()
t2

## pytest

In [ ]:
import ipytest

ipytest.autoconfig()

## bias update

In [ ]:
import torch

batch_size = 3
nudge_n = torch.rand((batch_size, 5))
free_n = torch.rand((batch_size, 5))
(nudge_n - free_n) * (nudge_n + free_n - 2 * torch.ones_like(free_n))  # (n-1)

In [ ]:
res = (nudge_n - free_n) * (nudge_n + free_n - 2 * torch.ones_like(free_n))  # (n-1)
res = res.mean(dim=0)

In [ ]:
res

In [ ]:
mean_nudge_n = torch.mean(nudge_n, dim=0)
mean_free_n = torch.mean(free_n, dim=0)
(mean_nudge_n - mean_free_n) * (
    mean_nudge_n + mean_free_n - 2 * torch.ones_like(mean_free_n)
)  # (n-1)

In [ ]:
torch.inner((nudge_n - free_n), (nudge_n + free_n - 2 * torch.ones_like(free_n)))  # (n-1)

## broadcasting

In [ ]:
import torch

A = torch.rand((2, 3))
v = torch.ones((3))  # row vector
print(A)
print(v)
A += v
print(A)

v = torch.ones((3))
print(A.T + v)  # error

In [ ]:
import torch

A = torch.rand((2, 3, 3))
iden = torch.eye(3)

print(A + iden)

In [ ]:
v = torch.arange(8).reshape(2, 4)
# repeat v along the last dimension
v.repeat(2, 1, 3)

In [ ]:
A = torch.ones((2, 4))

In [ ]:
A + v.float().mean(dim=0)

In [ ]:
v.float().mean(dim=0)

In [ ]:
v.pow(2).float().mean(dim=0).unsqueeze(-1).expand_as(A)

In [ ]:
lin = torch.nn.Linear(3, 4)

In [ ]:
lin.weight.shape

## einops

In [ ]:
import torch

x = torch.rand((4))  # batch_size x input_dim
W = torch.rand((3, 4))  # output_dim x input_dim

y = torch.einsum("...i,oi->...o", x, W)

In [ ]:
y.shape

## etc

In [ ]:
import torch.nn as nn

lyr1 = nn.Linear(1, 2, bias=False)

In [ ]:
hasattr(lyr1, "bias")

In [ ]:
import matplotlib.pyplot as plt

# plot 3d logsumexp
import torch

x = torch.linspace(-1, 1, 10)
y = torch.linspace(-1, 1, 10)
xy = torch.meshgrid(x, y)
logsumexp = torch.logsumexp(xy, dim=0)
plt.plot3d(xy, logsumexp)

In [ ]:
y = torch.rand((7))

In [ ]:
import torch
import torch.nn as nn

mdl = nn.Linear(1, 1, bias=True)

In [ ]:
mdl.bias

In [ ]:
list(mdl.named_parameters())

In [ ]:
# create empty list
W = torch.rand((40, 5000))
b = torch.randint(1, 5, (5000,))
print(W, b)

In [ ]:
# %%timeit
W / b

In [ ]:
D = torch.diag_embed(b).float()

In [ ]:
%%timeit
W @ D

In [ ]:
from src.utils import eqprop_util

In [ ]:
B = 64
In = Out = 1000
a = torch.rand((B, In))
b = torch.rand((B, Out))

In [ ]:
%%timeit
eqprop_util.deltaV(b, a)

In [ ]:
%run -c python -m foo.py cProfile

In [ ]:
import functools

import torch

from src.utils import eqprop_util

partial_fn = functools.partial(eqprop_util.rectifier_p3_i, Is=1e-6)
scrfn = torch.jit.script(functools.wraps(partial_fn), example_inputs=(torch.rand(2, 3),))

In [ ]:
%%timeit
scrfn(torch.rand(128, 3000))

In [ ]:
%%timeit
eqprop_util.rectifier_p3_i(torch.rand(128, 3000))

In [ ]:
li = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10)
li[-2:]

In [ ]:
import torch

from src.eqprop import eqprop_util

ots = eqprop_util.OTS()
V = torch.rand((10, 100))

In [ ]:
%%timeit
ots.a(V)

In [ ]:
ots.i

In [ ]:
@torch.jit.script
def a(
    V: torch.Tensor, Vth: float = 0.026, Vr: float = 0.9, Vl: float = 0.1, Is: float = 1e-8
) -> torch.Tensor:
    admittance = Is / Vth * (torch.exp((V - Vr) / Vth) + torch.exp((-V + Vl) / Vth))
    return admittance

In [ ]:
%%timeit
a(V)

## Im2Col (nn.Unfold)

In [ ]:
import torch

from src.models.components.vgg11 import VGG11Im2Col

model = VGG11Im2Col(
    hidden_channels=tuple((9 * torch.tensor([7, 7 * 2, 7 * 4, 7 * 8, 7 * 9])).tolist())
)

x = torch.rand((1, 3, 32, 32))
y = model(x)

In [ ]:
import torch

from src.models.components.vgg11 import VGG11avg, VGG11Im2Col

model1 = VGG11Im2Col(
    hidden_channels=tuple((9 * torch.tensor([7, 7 * 2, 7 * 4, 7 * 8, 7 * 9])).tolist())
)
model = VGG11avg(
    hidden_channels=tuple((9 * torch.tensor([7, 7 * 2, 7 * 4, 7 * 8, 7 * 9])).tolist())
)
# x = torch.rand((1, 3, 32, 32))
# y = model(x)

In [ ]:
model1.hidden_channels

In [ ]:
x = torch.rand((1, 3, 32, 32))
for m in model1.features.children():
    print(m)
    print(x.shape)
    x = m(x)

In [ ]:
import torch
import torch.nn as nn

unfold = nn.Unfold(kernel_size=(2, 3))
input = torch.randn(2, 5, 3, 4)
output = unfold(input)
# each patch contains 30 values (2x3=6 vectors, each of 5 channels)
# 4 blocks (2x3 kernels) in total in the 3x4 input
output.size()

# Convolution is equivalent with Unfold + Matrix Multiplication + Fold (or view to output shape)
inp = torch.randn(1, 3, 10, 12)
w = torch.randn(2, 3, 4, 5)
inp_unf = torch.nn.functional.unfold(inp, (4, 5))
out_unf = inp_unf.transpose(1, 2).matmul(w.view(w.size(0), -1).t()).transpose(1, 2)
out = torch.nn.functional.fold(out_unf, (7, 8), (1, 1))
# or equivalently (and avoiding a copy),
# out = out_unf.view(1, 2, 7, 8)
(torch.nn.functional.conv2d(inp, w) - out).abs().max()

In [ ]:
import torch
import torch.nn as nn

critertion = nn.MSELoss(reduction="sum")

y_hat = torch.arange(3).repeat(4, 1).float().requires_grad_(True)
y = torch.zeros((4, 3)).float()
loss = critertion(y_hat, y)

In [ ]:
loss.backward()

In [ ]:
y_hat.grad

# Numpy

In [ ]:
import numpy as np

li = [1, 2, 3]
arr = np.random.rand(*tuple(li))
len(arr.shape)

# EP

In [ ]:
from src.models.components.vgg11 import VGG11

model = VGG11()
[print(m) for m in model.features.children()]

In [ ]:
for name, param in model.named_parameters():
    print(name, param.shape)

# END